In [ ]:
!pip -q install onnxruntime

In [58]:
token = !oc whoami -t
token = token[0]

In [60]:
model_name = "jukebox"

In [32]:
import pandas as pd
import pickle
import json
import onnxruntime as rt
import numpy as np
import onnx

import requests
from urllib.parse import urljoin

In [12]:
song_properties = pd.read_parquet('https://github.com/rhoai-mlops/jukebox/raw/refs/heads/main/99-data_prep/song_properties.parquet')
song_properties=song_properties.head(1000)
X = song_properties[['is_explicit', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

In [13]:
with open('../2-dev_datascience/models/jukebox/1/artifacts/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

In [14]:
data = scaler.transform(X.values).tolist()

In [15]:
sess = rt.InferenceSession("../2-dev_datascience/models/jukebox/1/model.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: scaler.transform(X.values).astype(np.float32)}) 

In [16]:
training_data = {
    "model_name": model_name,
    "data_tag": "TRAINING",
    "request": {
        "inputs": [
            {
                "name": input_name,
                "shape": np.shape(data),
                "datatype": "FP32",
                "data": data
            }
        ]    
    },
    "response": {
        "model_name": model_name,
        "model_version": "1",
        "outputs": [
            {
                "name": output_name,
                "datatype": "FP32",
                "shape": np.shape(y_pred_temp[0]),
                "data": y_pred_temp[0].tolist()
            }
        ]
    }
}


In [41]:
base_url = "your-trustyai-service"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

In [ ]:
# Upload data
endpoint = "data/upload"
url = urljoin(base_url, endpoint)
response = requests.post(url, headers=headers, json=training_data)
print(response.text)

In [ ]:
# Change the input and output names
endpoint = "info/names"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "inputMapping": {
        "dense_5_input-0": "is_explicit",
        "dense_5_input-1": "duration_ms",
        "dense_5_input-2": "danceability",
        "dense_5_input-4": "energy",
        "dense_5_input-5": "key",
        "dense_5_input-6": "loudness",
        "dense_5_input-7": "mode",
        "dense_5_input-8": "speechiness",
        "dense_5_input-9": "acousticness",
        "dense_5_input-10": "instrumentalness",
        "dense_5_input-11": "liveness",
        "dense_5_input-12": "valence",
        "dense_5_input-13": "tempo"
    },
    # "outputMapping": {
    #     "dense_9-0": "Country Probability"
    # }
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

In [ ]:
# See if the change went through
endpoint = "/info"
url = urljoin(base_url, endpoint)
response = requests.get(url, headers=headers)
print(response.text)

In [ ]:
#Activate the metric
endpoint = "/metrics/drift/meanshift/request"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "referenceTag": "TRAINING"
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)